# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [104]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u




In [105]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [106]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [107]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)


[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [108]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [109]:
np.linalg.matrix_rank(a1)

6

In [110]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in matmul
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in matmul
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [111]:

def f(a):
    k = 0
    for n in reversed(range(2, N+1)):
        b = a[0:n, 0:n]
        if np.linalg.det(b) == 0:
            k += 1
    if k == 0:
        return True
    else:
        return False


In [112]:
print(f(a))
print(f(a1))

True
False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [132]:
def f(a):
    N = a.shape[0]
    U, L, P = a.copy(), np.eye(N), np.eye(N)
    for j in range(N-1): 
        for i in range(j, N):
                if U[i, j] != 0:
                    k = i
                    break
                    
        Pk, Pk[k, k], Pk[j, k] = np.eye(N), 0, 1
        Pk[k, j] = 1 
        U[[j, k],:] =U[[k, j],:]
        P, lam, gamma = Pk@P, np.eye(N), U[j+1:, j] / U[j, j]
        
        lam[j+1:, j] = -gamma
        U = lam @ U
        lam[j+1:, j] = gamma
        L = L @ lam 
        
    return L, U, P

print(f(a))
print('\n')
print(f(a1))

(array([[1.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 1.455, 1.   , 0.   , 0.   , 0.   ],
       [1.   , 1.714, 1.742, 1.   , 0.   , 0.   ],
       [1.   , 1.882, 2.276, 2.039, 1.   , 0.   ],
       [1.   , 2.   , 2.671, 2.944, 2.354, 1.   ]]), array([[ 3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,
         3.000e+00],
       [ 0.000e+00, -1.125e+00, -1.636e+00, -1.929e+00, -2.118e+00,
        -2.250e+00],
       [ 0.000e+00,  0.000e+00,  2.625e-01,  4.574e-01,  5.975e-01,
         7.013e-01],
       [ 0.000e+00,  1.110e-16,  0.000e+00, -2.197e-02, -4.480e-02,
        -6.469e-02],
       [ 0.000e+00, -2.819e-16,  0.000e+00,  0.000e+00,  8.080e-04,
         1.902e-03],
       [ 0.000e+00,  3.369e-16,  0.000e+00, -1.541e-18,  2.168e-19,
        -1.585e-05]]), array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0.,

In [128]:
def g(L, u, P):
    return P@L@u
#Проверим, что обратное преобразование даёт исходные матрицы, разница должна быть нулевой (ну или очень маленькой)
print(g(*f(a))-a)
print('\n')
print(g(*f(a1))-a1)

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]]


[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 3.000e+00  1.364e+00  8.824e-01  6.522e-01  5.172e-01  4.286e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  1.665e-16  5.551e-17]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]]
